In [1]:
#change runtime to A100 GPU, then run cell:
!pip install numpy
from numpy.random import seed #fix random seed
import os, glob
seed(123)

### 1. Load data
* Load data into two folders: one for split stems (2-stem) and one for effects (4-stem)
* Mount to google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 2.1 Models- Spleeter



In [ ]:
#spleeter needs oldeer version of python, use pyenv to downnload it
!git clone https://github.com/pyenv/pyenv.git ~/.pyenv
import os

# Set the root for pyenv
os.environ['PYENV_ROOT'] = os.path.expanduser("~/.pyenv")
# Prepend pyenv's bin folder to PATH
os.environ['PATH'] = os.environ['PYENV_ROOT'] + '/bin:' + os.environ['PATH']

# Confirm pyenv is callable
!pyenv --version
!apt-get install libffi-dev
!pyenv install 3.8.13
!pyenv global 3.8.13
!pyenv exec pip install spleeter
!pyenv exec spleeter

2-stem separation

In [ ]:
%%bash
pyenv exec python - << 'EOF'
from spleeter.separator import Separator
import os, glob

dataset_2 =  "/content/drive/My Drive/MIR_Project/Paired_Stems"
data_home = dataset_2
output_home = "/content/drive/My Drive/MIR_Project/Spleeter_Outputs/Paired_Stems"
os.makedirs(output_home, exist_ok=True)

# load model
separator = Separator("spleeter:2stems")

# get audio files
audio_files = glob.glob(os.path.join(data_home, "**/*.wav"), recursive=True)
print(f"Found {len(audio_files)} files.")

# run model
for audio_path in audio_files:
    try:
        print("Separating:", audio_path)
        song_folder = os.path.basename(os.path.dirname(audio_path))

        song_out_dir = os.path.join(output_home, song_folder)
        os.makedirs(song_out_dir, exist_ok=True)

        separator.separate_to_file(audio_path, song_out_dir)

    except Exception as e:
        print("Failed on", audio_path, "->", e)
EOF

4-stem separation (run original files along with processed)

In [ ]:
%%bash
pyenv exec python - << 'EOF'
from spleeter.separator import Separator
import os, glob

dataset_4 = "/content/drive/My Drive/MIR_Project/Effects"
data_home = dataset_4
output_home = "/content/drive/My Drive/MIR_Project/Spleeter_Outputs/Effects"
os.makedirs(output_home, exist_ok=True)

# load model
separator = Separator("spleeter:4stems")

# get audio files
audio_files = glob.glob(os.path.join(data_home, "**/*.wav"), recursive=True)
print(f"Found {len(audio_files)} files for 4-stem separation.")

# run model
for audio_path in audio_files:
    try:
        print("Separating (4 stems):", audio_path)
        separator.separate_to_file(audio_path, output_home)
    except Exception as e:
        print("Failed on", audio_path, "->", e)
EOF

### 2.2 Models- Hybrid Demucs


In [ ]:
pip install torchcodec
import torch, torchaudio
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB_PLUS

#pick whether to run model on GPU or CPU depending on availability.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)


2-stems

In [ ]:
import os, glob
import torchaudio
import torch

# paths
dataset_2 = "/content/drive/My Drive/MIR_Project/Paired_Stems"
data_home = dataset_2
output_home  = "/content/drive/My Drive/MIR_Project/Demucs_Outputs/Paired_Stems"
os.makedirs(output_home, exist_ok=True)

# load model
bundle = HDEMUCS_HIGH_MUSDB_PLUS
model = bundle.get_model().to(device).eval()
labels = ["drums", "bass", "other", "vocals"]
sr_target = bundle.sample_rate

audio_files = glob.glob(os.path.join(data_home, "**/*.wav"), recursive=True)
print(f"Found {len(audio_files)} .wav files.")

processed_count_demucs_2stems = 0

# run model
for audio_path in audio_files:
    try:
        print("Separating:", audio_path)

        waveform, sr = torchaudio.load(audio_path)

        if sr != sr_target:
            waveform = torchaudio.functional.resample(waveform, sr, sr_target)

        if waveform.size(0) == 1:
            waveform = waveform.repeat(2, 1)

        with torch.inference_mode():
            sources4 = model(waveform.unsqueeze(0).to(device))

        if sources4.dim() == 4:
            sources4 = sources4[0]

        drums  = sources4[0]
        bass   = sources4[1]
        other  = sources4[2]
        vocals = sources4[3]

        # Merge to 2 stems
        accompaniment = drums + bass + other
        sources2 = torch.stack([accompaniment, vocals], dim=0)
        labels2  = ["accompaniment", "vocals"]

        #Save
        track = os.path.splitext(os.path.basename(audio_path))[0]
        track_dir = os.path.join(output_home, track)
        os.makedirs(track_dir, exist_ok=True)

        for i, name in enumerate(labels2):
            torchaudio.save(
                os.path.join(track_dir, f"{name}.wav"),
                sources2[i].cpu(),
                sr_target
            )

        processed_count_demucs_2stems += 1

    except Exception as e:
        print("Failed on", audio_path, "->", e)

print(f"Successfully processed {processed_count_demucs_2stems} audio files for Demucs 2-stems separation.")

4-stems (run original files along with processed)

In [ ]:
#paths
dataset_4 = "/content/drive/My Drive/MIR_Project/Effects" #run again with original
data_home = dataset_4
output_home  = "/content/drive/My Drive/MIR_Project/Demucs_Outputs/Effects" #run again with original
os.makedirs(output_home, exist_ok=True)

#load model
bundle = HDEMUCS_HIGH_MUSDB_PLUS
model = bundle.get_model().to(device).eval()
labels = ["drums", "bass", "other", "vocals"]
sr_target = bundle.sample_rate      # 44100

#get audio files
audio_files = glob.glob(os.path.join(data_home, "**/*.wav"), recursive=True)
print(f"Found {len(audio_files)} files.")

processed_count_demucs_4stems = 0

# run model
for audio_path in audio_files:
    try:
        print("Separating:", audio_path)

        waveform, sr = torchaudio.load(audio_path)

        if sr != sr_target:
            waveform = torchaudio.functional.resample(waveform, sr, sr_target)

        if waveform.size(0) == 1:
            waveform = waveform.repeat(2, 1)

        with torch.inference_mode():
            # Add a batch dimension here: [channels, frames] -> [1, channels, frames]
            sources = model(waveform.unsqueeze(0).to(device))  # [4, channels, time]
            #if using chunking, replace with:
            # sources = separate_sources(model, waveform[None].to(device))[0]

        track = os.path.splitext(os.path.basename(audio_path))[0]
        track_dir = os.path.join(output_home, track)
        os.makedirs(track_dir, exist_ok=True)

        for i, name in enumerate(labels):
            torchaudio.save(
                os.path.join(track_dir, f"{name}.wav"),
                sources[0][i].cpu(), # Access the first (and only) item in the batch
                sr_target
            )
        processed_count_demucs_4stems += 1

    except Exception as e:
        print("Failed on", audio_path, "->", e)
print(f"Successfully processed {processed_count_demucs_4stems} audio files for Demucs 4-stems separation.")

#''''
#***If that ^ takes up too much memory, then do this instead (chunking)*****

#from torchaudio.transforms import Fade
#import torch

#def separate_sources(model, mix, segment=10.0, overlap=0.1, device=None):
#    if device is None:
#        device = mix.device
#    else:
#        device = torch.device(device)

#    batch, channels, length = mix.shape
#    sample_rate = 44100  # bundle.sample_rate
#    chunk_len = int(sample_rate * segment * (1 + overlap))
#    start = 0
#    end = chunk_len
#    overlap_frames = int(overlap * sample_rate)
#    fade = Fade(fade_in_len=0, fade_out_len=overlap_frames, fade_shape="linear")

#    final = torch.zeros(batch, len(model.sources), channels, length, device=device)

#    while start < length - overlap_frames:
#        chunk = mix[:, :, start:end]
#        with torch.no_grad():
#            out = model(chunk)
#        out = fade(out)
#        final[:, :, :, start:end] += out

#        start += chunk_len - overlap_frames
#        end = start + chunk_len

#        if end >= length:
#            end = length
#            fade.fade_out_len = 0

#    return final

#''''''

### 2.3 Models- Evaluation

###3.1 Experiment 1